In [1]:
import sys

sys.path.append("..")  # Adds higher directory to python modules path.
import os
import numpy as np
import polars as pl
from src import RASPRoutines
from src import AnalysisFunctions
import matplotlib.pyplot as plt
A_F = AnalysisFunctions.Analysis_Functions()

RASP = RASPRoutines.RASP_Routines()

In [25]:
""" the command below will re-run colocalisation analysis on already analysed data

analysis_file: an analysis file .csv

threshold: a photon threshold

protein_string: a string for images with protein in

cell_string: a string for cellular images

"""

analysis_folder = r"S:\ASAP_Imaging_Data\Main_Survey\20241105_oligomers_in_cells\caudate\microglia\PD_analysis"
analysis_file = os.path.abspath(
    r"S:\ASAP_Imaging_Data\Main_Survey\20241105_oligomers_in_cells\caudate\microglia\PD_analysis\spot_analysis_0_abovephotonthreshold.csv"
)  # example file, replace as you wish. Always input as the absolute path of a raw string
threshold = 0.0
protein_string = "C1"
cell_string = "C0"
imtype = '.tif'
lower_cell_size_threshold = 2000.0
analysis_data = pl.read_csv(analysis_file)
z_project_first = [True, True]

cell_punctum_analysis_file = r"S:\ASAP_Imaging_Data\Main_Survey\20241105_oligomers_in_cells\caudate\microglia\PD_analysis\single_cell_coincidence_mincellsize_2000_photonthreshold_525p1_photons_combined_threshold_percentile_90_k_4_outliersremoved_abovethreshold.csv"
cell_punctum_analysis = pl.read_csv(cell_punctum_analysis_file)
cell_punctum_analysis = cell_punctum_analysis.filter(pl.col("puncta_cell_likelihood") != np.NAN)

In [64]:
image_filenames = np.unique(cell_punctum_analysis["image_filename"].to_numpy())

In [66]:
n_cells_per_image = np.zeros(len(image_filenames))
for i, filename in enumerate(image_filenames):
    n_cells_per_image[i] = len(cell_punctum_analysis.filter(pl.col("image_filename") == filename)["area/pixels"].to_numpy())

In [69]:
# we can now use various plotting commands to select a z-slice and plot the cell mask vs number of puncta
import polars as pl
from src import IOFunctions

IO = IOFunctions.IO_Functions()

from src import AnalysisFunctions

A_F = AnalysisFunctions.Analysis_Functions()
import os

cell_file = np.argmax(n_cells_per_image)
file = image_filenames[cell_file]  # we take a file we're interested in
# now find the corresponding cell mask
mask_file = os.path.join(
    analysis_folder,
    str(
        os.path.split(file)[-1].split(imtype)[0].split(protein_string)[0]
        + cell_string
        + "_cellMask.tiff"
    ),
)

# and the corresponding raw cell data
raw_cell_file = str(file.split(imtype)[0].split(protein_string)[0]+cell_string+imtype)

In [70]:
# read the raw images in
puncta_image = IO.read_tiff_tophotons(file)
cell_mask = IO.read_tiff(mask_file)
cell_image = IO.read_tiff_tophotons(raw_cell_file)

In [74]:
oned_cell_mask, pil, centroids, areas = A_F.threshold_cell_areas(
        cell_mask,
        lower_cell_size_threshold=lower_cell_size_threshold,
        z_project=[True, True],
    )
oned_puncta_image = np.sum(puncta_image, axis=-1)
oned_cell_image = np.sum(cell_image, axis=-1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from src import PlottingFunctions

plotter = PlottingFunctions.Plotter()

fig, axs = plotter.two_column_plot(
    ncolumns=3, nrows=2, widthratio=[1, 1, 1], heightratio=[1, 1]
)

axs[0, 0] = plotter.image_scatter_plot(
    axs=axs[0, 0],
    xdata=puncta["x"],
    ydata=puncta["y"],
    data=cell_mask[:, :, z_plane - 1],
    cbarlabel=r"cell mask",
    s=0.01,
    lws=0.25,
)

axs[0, 1] = plotter.scatter_plot(
    axs=axs[0, 1],
    x=analysis["area/pixels"].to_numpy() * (0.11**2),
    y=analysis[:, 5].to_numpy(),
    s=2,
    facecolor="red",
    edgecolor=None,
    label=r"$n$ puncta > 1000 photons",
)
axs[0, 1] = plotter.scatter_plot(
    axs=axs[0, 1],
    x=analysis["area/pixels"].to_numpy() * (0.11**2),
    y=analysis[:, 4].to_numpy(),
    s=2,
    facecolor="black",
    edgecolor=None,
    label=r"$n$ puncta <= 1000 photons",
    alpha=0.5,
    xaxislabel=r"cell area ($\mu$m$^2$)",
    yaxislabel="number of puncta",
)
axs[0, 1].legend(loc="upper right", fontsize=6)
xdiff = np.diff(axs[0, 1].get_xlim())[0]
ydiff = np.diff(axs[0, 1].get_ylim())[0]
axs[0, 1].set_aspect(0.9 * (xdiff / ydiff), adjustable="box")

y = analysis["n_puncta_in_cell_ratio_aboveandbelow"].to_numpy()
inds = np.isfinite(y)
y = y[inds]
x = analysis["area/pixels"].to_numpy()[inds] * (0.11**2)
axs[0, 2] = plotter.scatter_plot(
    axs=axs[0, 2],
    x=x,
    y=y,
    s=5,
    facecolor="red",
    edgecolor=None,
    yaxislabel="ratio of bright/dim puncta",
    xaxislabel=r"cell area ($\mu$m$^2$)",
)
axs[0, 2].set_ylim([0, 1.05])
xdiff = np.diff(axs[0, 1].get_xlim())[0]
axs[0, 2].set_aspect(0.9 * (xdiff / 1.05), adjustable="box")

axs[1, 0] = plotter.image_plot(
    axs=axs[1, 0], data=cell_mask_toplot_AT, cbarlabel=r"$n$ puncta > 1000 photons"
)
axs[1, 1] = plotter.image_plot(
    axs=axs[1, 1], data=cell_mask_toplot_UT, cbarlabel=r"$n$ puncta <= 1000 photons"
)
axs[1, 2] = plotter.image_plot(
    axs=axs[1, 2], data=cell_mask_toplot_R, cbarlabel=r"ratio of bright/dim puncta"
)

plt.tight_layout(h_pad=-15)

plt.show()